# Preprocessing

1. Download the `ks_wells.zip` archive from [Kansas Geological Survey](https://www.kgs.ku.edu/PRS/Ora_Archive/ks_wells.zip).

2. Download the `ks_wells.txt` file to get access to LAS files URLs.

3. Create a folder named `data` in your project directory.

4. Save the `ks_wells.txt` file in the `data` folder.




In [ ]:
# Unzip files in their respective folders
from src.unzip_files import unzip_files_KGS 

source_folder = '../data/zip_files'
destination_folder = '../data/v1.0_raw_data'

unzip_files_KGS(source_folder, destination_folder)

In [ ]:
# Download las files from KGS
from src.download_las_files import download_and_organize_las_files

source_folder = '../data/v1.0_raw_data'  
destination_folder = '../data/v2.0_zip_files'

# File to map las file names to urls
database_path = '../data/zip_files/ks_las_files.txt'  

# Download and organize las files
download_and_organize_las_files(source_folder, destination_folder, database_path)

Jones Canyon Southeast |                                                  
Medicine Lodge North   |                                                  
Schaben                |                                                  
Marcotte               |                                                  
Zenith Peace Creek     |                                                  
Cutter                 |                                                  
Thrall Aagard          |                                                  
Wellington             |                                                  
Burrton                |                                                  
Kraft Prusa            |                                                  
El Dorado              |                                                  
Webster                |                                                  
Bemis Shutts           |                                                  
Hall Gurney            | 

In [1]:
from src.process_all_las_files import LASFileProcessor

source_folder = '../data/v2.0_zip_files'
destination_folder = '../data/v3.0_las_files'
csv_folder = '../data/v1.0_raw_data'

LASFileProcessor(source_folder, destination_folder, csv_folder).process_las_files()

Arroyo                 |                                                  
Bemis Shutts           |                                                  
Bradshaw Gas Area      |                                                  
Burrton                |                                                  
Cooper                 |                                                  
Cutter                 |                                                  
East Fork              |                                                  
El Dorado              |                                                  
Garnett Shoestring     |                                                  
Hall Gurney            |                                                  
Jones Canyon Southeast |                                                  
Kraft Prusa            |                                                  
Marcotte               |                                                  
Medicine Lodge North   | 

## Select Field

In [1]:
import sys
import os

# Ensure the src and ux_ui directories are in the Python path
sys.path.append(os.path.abspath('./src'))
sys.path.append(os.path.abspath('./ux_ui'))

from ux_ui.load_and_select_field_ui import load_and_select_field_ui
from src.project_manager import ProjectManager  # Correctly import the ProjectManager class

# Initialize the ProjectManager with the base directory
base_directory = '../data/v3.0_las_files'
project_manager_instance = ProjectManager(base_directory)

# Load and display the UI
load_and_select_field_ui(project_manager_instance)

## Set stats by curve

In [3]:
from ux_ui.display_curve_descriptions import display_curve_descriptions

# Display the curve descriptions after loading the field
display_curve_descriptions(project_manager_instance)

Box(children=(HTML(value='\n        <style>\n            table {\n                border-collapse: collapse;\n…

In [3]:
from ux_ui.curve_selection_ui import curve_selection_and_grouping_ui

curve_selection_and_grouping_ui(project_manager_instance)
# Group in order for future plotting, make sure attributes selected_curves and standardized_curve_mapping 
# are corretly written checkin project manager instances object

HTML(value='\n    <style>\n        .title {\n            text-align: center;\n            font-size: 20px;\n  …

In [5]:
from ux_ui.display_statistics import display_statistics

display_statistics(project_manager_instance)

In [2]:
from ux_ui.plots_by_curve import create_statistics_plots_by_curve_ui

project_manager_instance.selected_curves = ['RILD', 'SP', 'MI', 'DPOR', 'DCAL', 'DT', 'CILD', 'MN', 'SPOR', 'RILM', 'GR', 'RHOB', 'MCAL', 'SCAL']

create_statistics_plots_by_curve_ui(project_manager_instance)

## Set stats by field.

In [1]:
from ux_ui.missingno_ui import create_missing_data_uib

project_manager_instance.selected_curves = ['RILD', 'SP', 'MI', 'DPOR', 'DCAL', 'DT', 'CILD', 'MN', 'SPOR', 'RILM', 'GR', 'RHOB', 'MCAL', 'SCAL']

create_missing_data_ui(project_manager_instance)
# Load report generate all individual plots saving a picture in folders according to well name, it could take several minutes.

ImportError: cannot import name 'create_missing_data_uib' from 'ux_ui.missingno_ui' (/home/pokinux/Kansas/code/ux_ui/missingno_ui.py)

In [1]:
# from src.project_manager import ProjectManager 

# base_directory = '../data/v3.0_las_files'
# project_manager_instance = ProjectManager(base_directory)
# project_manager_instance.selected_field = 'Jones Canyon Southeast'
# project_manager_instance.load_selected_field
# project_manager_instance.project

In [1]:
import sys
import os

# Ensure the src and ux_ui directories are in the Python path
sys.path.append(os.path.abspath('./src'))
sys.path.append(os.path.abspath('./ux_ui'))

from ux_ui.load_and_select_field_ui import load_and_select_field_ui
from src.project_manager import ProjectManager  # Correctly import the ProjectManager class

# Initialize the ProjectManager with the base directory
base_directory = '../data/v3.0_las_files'
project_manager_instance = ProjectManager(base_directory)

# Load and display the UI
load_and_select_field_ui(project_manager_instance)

In [3]:
from ux_ui.create_log_plot_ui import well_plots_ui

project_manager_instance.selected_curves = ['DEPT','RILD', 'SP', 'MI', 'DPOR', 'DCAL', 'DT', 'CILD', 'MN', 'SPOR', 'RILM', 'GR', 'RHOB', 'MCAL', 'SCAL']
project_manager_instance.standardized_curve_mapping = {
    'Cali': ['DCAL', 'SCAL', 'MCAL'],
    'GR-SP': ['GR', 'SP'],
    'RIL': ['RILM', 'RILD'],
    'Micro': ['MN', 'MI'],
    'Density': ['RHOB', 'RHOC', 'DPOR'],
    'Sonic': ['SPOR', 'ITT', 'DT'],
    }

# Assuming project_manager_instance is already defined
well_plots_ui(project_manager_instance)

In [2]:
project_manager_instance.formation_data




{"Mears 'A' 1-9": [(295.0, 3088.0, 'Unknown'),
  (3088.0, 3122.0, 'Stone Corral Anhydrite'),
  (3656.0, 3876.0, 'Foraker'),
  (3876.0, 4010.0, 'Topeka'),
  (4010.0, 4092.0, 'Oread'),
  (4092.0, 4148.0, 'Lansing A'),
  (4148.0, 4202.0, 'Lansing B'),
  (4202.0, 4246.0, 'Lansing C'),
  (4246.0, 4294.0, 'Lansing D'),
  (4294.0, 4336.0, 'Lansing E'),
  (4336.0, 4486.0, 'Lansing F'),
  (4486.0, 4544.0, 'Pawnee'),
  (4544.0, 4806.0, 'Cherokee'),
  (4806.0, 4968.0, 'Mississippian'),
  (4968.0, 5048.0, 'Arbuckle'),
  (5048.0, 5107.0, 'Reagan Sand'),
  (5107.0, 5116.0, 'Granite Wash'),
  (5116.0, 5090.5, 'Granite')],
 'Theron 2-10': [(-2.0, 3048.0, 'Unknown'),
  (3048.0, 3079.0, 'Anhydrite'),
  (3500.0, 3563.0, 'Neva'),
  (3563.0, 3608.0, 'Red Eagle'),
  (3608.0, 3831.0, 'Foraker'),
  (3831.0, 3965.0, 'Topeka'),
  (3965.0, 4047.0, 'Oread'),
  (4047.0, 4316.0, 'Lansing-Kansas City'),
  (4263.0, 4436.0, 'Stark'),
  (4436.0, 4522.0, 'Pawnee'),
  (4522.0, 4625.5, 'Cherokee')],
 'Patti 1-10': [(-4.0,

Unique formations:
- Anhydrite
- Arbuckle
- Blaine
- Carlile Shale
- Chase
- Cherokee
- Cherokee Shale
- Cheyenne
- Dakota
- Deer Creek
- Foraker
- Fort Hays Limestone Member
- Granite
- Granite Wash
- Heebner Shale
- LKC B
- LKC C
- LKC D
- LKC E
- LKC F
- Lansing
- Lansing A
- Lansing B
- Lansing C
- Lansing D
- Lansing E
- Lansing F
- Lansing-Kansas City
- Leavenworth Lime
- Mississippian
- Neva
- Niobrara
- Oread
- Pawnee
- Reagan Sand
- Red Eagle
- Stark
- Stark Shale
- Stone Corral Anhydrite
- Topeka
- Wabaunsee


In [6]:
project_manager_instance.project[0].header

,original_mnemonic,mnemonic,unit,value,descr,section
0,VERS,VERS,,2.0,CWLS log ASCII Standard -VERSION 2.0,Version
1,WRAP,WRAP,,NO,One line per depth step,Version
2,STRT,STRT,FT,295.0,,Well
3,STOP,STOP,FT,5090.5,,Well
4,STEP,STEP,FT,0.5,,Well
5,NULL,NULL,,-999.25,,Well
6,COMP,COMP,,BEREXCO LLC,CURR_OPERATOR,Well
7,WELL,WELL,,1-9,WELL_NAME,Well
8,FLD,FLD,,Jones Canyon Southeast,FIELD_NAME,Well
9,LOC,LOC,,"T1S R37W, Sec. 9, SW NE SE NE",LOCATION,Well


#### To fix list

* Saving las file
    - Set-up format in curve information
    - set well name in header las file
    - Merge separate las files for same well

* Select curves and mapping
    - white empty html area when running the code.

* Statistics plots by curve
    - set dimension for plotting area while selected several wells

* Missing data analysis 
    - Save file twice


#### To do list:

* Select Field
    - add map
    - add well location
    - add field boundaries

* Statistics
    - Process statistics for all curves when no curves selected for project
    - Define thresholds or criteria for data filtering
        class ProjectManager.descriptive_statistics
        valid_range = (-1000, 10000) --> actual value / add IntRangeSlider in UI to set range

* Missingno plots
    - Optimize loading plots
    - Create a summarize report plot
    
* Curve selection panel
* Plots

* Others    
    - LLM with RAG to check formation.
    - Check Multithreading/Processing Debugging in Monitor System Resources
